In [17]:
%matplotlib inline

Word-level language modeling RNN
==================================================================


## 선정이유
: 문장을 새로 만든다는게 신기해서 한번 해보고 싶었습니다.

## 소개
------------
This example trains a multi-layer RNN (Elman, GRU, or LSTM) on a language modeling task. By default, the training script uses the Wikitext-2 dataset, provided. The trained model can then be used by the generate script to generate new text.


## 필요
-----------
pytorch 1.3이상

0. 필요한 라이브러리를 불러옵니다.

In [3]:
# imports
import os
from io import open
import time

import torch
import torch.nn as nn     # torch neural network 패키지를 가져옵니다.
import torch.nn.functional as F

1. 모델을 정의합니다.

Dropout: https://laonple.blog.me/220542170499
Embedding: https://blog.naver.com/limitsinx/221598890687

In [4]:
class LSTMModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(LSTMModel, self).__init__()
        self.drop = nn.Dropout(dropout)  # 랜덤하게 노드를 무시하여 오버피팅을 피하는 효과를 갖습니다. 또한 특정 노드의 가중치가
                                         # 너무 큰 영향을 주는 상황을 방지하게 해줍니다.
        self.encoder = nn.Embedding(ntoken, ninp) # 단어를 저차원(2~3) 실수 벡터로 표현합니다. 기존에 one-hot encoding은 모든 단어를 1또는 0
                                                  # 으로 이루어진 벡터로 표현해서 단어의 수가 늘수록 벡터의 차원이 커지는 단점이 있어
                                                  # 이를 개선한 방법입니다. 또한 단어간 거리를 측정 가능하여 단어간 유사도를 알 수 있습니다.
        self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout) 
        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output)
        return decoded, hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        return (weight.new_zeros(self.nlayers, bsz, self.nhid),
                weight.new_zeros(self.nlayers, bsz, self.nhid))

2. 텍스트 데이터를 불러옵니다.

In [17]:
help(Dictionary())

Help on Dictionary in module __main__ object:

class Dictionary(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __len__(self)
 |  
 |  add_word(self, word)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [6]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)


class Corpus(object):
    def __init__(self, path):
        self.dictionary = Dictionary()
        self.train = self.tokenize(os.path.join(path, 'train.txt'))
        self.valid = self.tokenize(os.path.join(path, 'valid.txt'))
        self.test = self.tokenize(os.path.join(path, 'test.txt'))    #각 멤버에 텍스트 데이터를 초기화시킵니다.

    def tokenize(self, path):
        """Tokenizes a text file."""
        assert os.path.exists(path)
        # Add words to the dictionary
        with open(path, 'r', encoding="utf8") as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    self.dictionary.add_word(word)

        # Tokenize file content
        with open(path, 'r', encoding="utf8") as f:
            idss = []
            for line in f:
                words = line.split() + ['<eos>']
                ids = []
                for word in words:
                    ids.append(self.dictionary.word2idx[word])
                idss.append(torch.tensor(ids).type(torch.int64))
            ids = torch.cat(idss)

        return ids

model_data_filepath = 'data/'

corpus = Corpus(model_data_filepath + 'wikitext-2')

   2-2. 학습

In [ ]:
!python main.py --epochs 5

3. 학습된 모델을 불러옵니다.

In [4]:
ntokens = len(corpus.dictionary)

model = LSTMModel(
    ntoken = ntokens,
    ninp = 512,
    nhid = 256,
    nlayers = 5,
)

model.load_state_dict(
    torch.load(
        model_data_filepath + 'word_language_model_quantize.pth',
        map_location=torch.device('cpu')
        )
    )

model.eval()
print(model)

LSTMModel(
  (drop): Dropout(p=0.5, inplace=False)
  (encoder): Embedding(33278, 512)
  (rnn): LSTM(512, 256, num_layers=5, dropout=0.5)
  (decoder): Linear(in_features=256, out_features=33278, bias=True)
)


In [5]:
input_ = torch.randint(ntokens, (1, 1), dtype=torch.long)
hidden = model.init_hidden(1)
temperature = 1.0
num_words = 1000

with open(model_data_filepath + 'out.txt', 'w') as outf:
    with torch.no_grad():  # no tracking history
        for i in range(num_words):
            output, hidden = model(input_, hidden)
            word_weights = output.squeeze().div(temperature).exp().cpu()
            word_idx = torch.multinomial(word_weights, 1)[0]
            input_.fill_(word_idx)

            word = corpus.dictionary.idx2word[word_idx]

            outf.write(str(word.encode('utf-8')) + ('\n' if i % 20 == 19 else ' '))

            if i % 100 == 0:
                print('| Generated {}/{} words'.format(i, 1000))

with open(model_data_filepath + 'out.txt', 'r') as outf:
    all_output = outf.read()
    print(all_output)

| Generated 0/1000 words
| Generated 100/1000 words
| Generated 200/1000 words
| Generated 300/1000 words
| Generated 400/1000 words
| Generated 500/1000 words
| Generated 600/1000 words
| Generated 700/1000 words
| Generated 800/1000 words
| Generated 900/1000 words
b'<unk>' b'Intelligence' b'as' b'king' b'Wilhelm' b'<unk>' b'Leo' b'Richard' b',' b'leaves' b'at' b'its' b'Loutit' b'.' b'In' b'the' b'context' b'of' b'Mikhail' b'@-@'
b'Admiral' b',' b'it' b'is' b'an' b'member' b'of' b'that' b'of' b'Omar' b"'s" b'supporters' b'Edith' b'role' b',' b'which' b'could' b'have' b'also' b'moved'
b'longer' b'to' b'do' b'so' b'mouth' b'for' b'Swedish' b'scenes' b'.' b'Both' b'in' b'his' b'microscope' b'and' b'the' b'creation' b'of' b'other' b'detail' b','
b'such' b'as' b'binary' b'droppings' b'on' b'the' b'map' b',' b'was' b'assigned' b'for' b'trees' b'.' b'It' b'does' b'not' b'sell' b'them' b'to' b'be'
b'placed' b'back' b'to' b'<unk>' b'General' b'Armstrong' b'720' b'then' b'species' b',' b'and' 

It's no GPT-2, but it looks like the model has started to learn the structure of
language!

We're almost ready to demonstrate dynamic quantization. We just need to define a few more
helper functions:



In [6]:
bptt = 25
criterion = nn.CrossEntropyLoss()
eval_batch_size = 1

# create test data set
def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    return data.view(bsz, -1).t().contiguous()

test_data = batchify(corpus.test, eval_batch_size)

# Evaluation functions
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

def repackage_hidden(h):
  """Wraps hidden states in new Tensors, to detach them from their history."""

  if isinstance(h, torch.Tensor):
      return h.detach()
  else:
      return tuple(repackage_hidden(v) for v in h)

def evaluate(model_, data_source):
    # Turn on evaluation mode which disables dropout.
    model_.eval()
    total_loss = 0.
    hidden = model_.init_hidden(eval_batch_size)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            output, hidden = model_(data, hidden)
            hidden = repackage_hidden(hidden)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

4. Test dynamic quantization
----------------------------

Finally, we can call ``torch.quantization.quantize_dynamic`` on the model!
Specifically,

- We specify that we want the ``nn.LSTM`` and ``nn.Linear`` modules in our
  model to be quantized
- We specify that we want weights to be converted to ``int8`` values



In [14]:
import torch.quantization

quantized_model = torch.quantization.quantize_dynamic(
    model, {nn.LSTM, nn.Linear}, dtype=torch.qint8
)
print(quantized_model)

RuntimeError: Didn't find engine for operation quantized::linear_prepack NoQEngine (operator () at ..\aten\src\ATen\native\quantized\cpu\qlinear_prepack.cpp:202)
(no backtrace available)

The model looks the same; how has this benefited us? First, we see a
significant reduction in model size:



In [15]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

print_size_of_model(model)
print_size_of_model(quantized_model)

Size (MB): 113.941528


NameError: name 'quantized_model' is not defined

Second, we see faster inference time, with no difference in evaluation loss:

Note: we number of threads to one for single threaded comparison, since quantized
models run single threaded.



In [16]:
torch.set_num_threads(1)

def time_model_evaluation(model, test_data):
    s = time.time()
    loss = evaluate(model, test_data)
    elapsed = time.time() - s
    print('''loss: {0:.3f}\nelapsed time (seconds): {1:.1f}'''.format(loss, elapsed))

time_model_evaluation(model, test_data)
time_model_evaluation(quantized_model, test_data)

loss: 5.167
elapsed time (seconds): 481.9


NameError: name 'quantized_model' is not defined

Running this locally on a MacBook Pro, without quantization, inference takes about 200 seconds,
and with quantization it takes just about 100 seconds.

Conclusion
----------

Dynamic quantization can be an easy way to reduce model size while only
having a limited effect on accuracy.

Thanks for reading! As always, we welcome any feedback, so please create an issue
`here <https://github.com/pytorch/pytorch/issues>`_ if you have any.

